# Homework 2 - Credit Analytics
Submission by Dennis Goldenberg

In [3]:
import numpy as np
import pandas as pd

## $\underline{\text{Question 1}}$

For this question: Read defaultswap.pdf

## $\underline{\text{Question 2}}$

For this question: read creditportfoliocalculation.pdf

## $\underline{\text{Question 3}}$

For this question: read understandingcopulas.pdf

## $\underline{\text{Question 4}}$

For this question: read AnOverViewofCopulaCDOVerion-6.pdf

## $\underline{\text{Question 5}}$

For this question: read creditportfoliocalculation.pdf

## $\underline{\text{Question 6}}$

For this question: read AnOverViewofCopulaCDOVerion-6.pdf